In [1]:
# !pip install https://github.com/pyg-team/pytorch_geometric.git

In [2]:
# !git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [28]:
%cd hackaton/

[Errno 2] No such file or directory: 'hackaton/'
/home/haislich/Documents/noisy_labels/hackaton


In [4]:
# !gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [5]:
!ls -lh datasets

total 16K
drwxr-xr-x 2 haislich haislich 4,0K 25 mag 11.19 A
drwxr-xr-x 2 haislich haislich 4,0K 25 mag 11.19 B
drwxr-xr-x 2 haislich haislich 4,0K 25 mag 11.19 C
drwxr-xr-x 2 haislich haislich 4,0K 25 mag 11.19 D


In [6]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


In [7]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [8]:
def train(data_loader, model, optimizer, criterion, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [65]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [54]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [11]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [20]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = ""#input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [58]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)", default = "./datasets/B/train.json.gz")
    args['test_path'] = get_user_input("Path to the test dataset",default = "./datasets/B/test.json.gz")
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [59]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Arguments received:
train_path: ./datasets/B/train.json.gz
test_path: ./datasets/B/test.json.gz
num_checkpoints: None
device: 1
gnn: gin
drop_ratio: 0.0
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 10
baseline_mode: 1
noise_prob: 0.2


In [60]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [61]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    criterion = NoisyCrossEntropyLoss(args.noise_prob)
else:
    criterion = torch.nn.CrossEntropyLoss()

In [62]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [63]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [64]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    
    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer, criterion, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}"),
            current_epoch=epoch
        )

        val_loss,val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating eval graphs:   6%|▌         | 2/35 [00:00<00:02, 14.36batch/s]

DataBatch(edge_index=[2, 135416], edge_attr=[135416, 7], y=[32], num_nodes=7817, x=[7817], batch=[7817], ptr=[33])
pred=tensor([4, 4, 4, 4, 4, 4, 0, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 4, 4,
        4, 0, 4, 4, 0, 4, 4, 4], device='cuda:0'), data.y=tensor([4, 0, 1, 1, 2, 4, 4, 3, 1, 4, 4, 0, 5, 3, 0, 2, 3, 1, 0, 4, 1, 4, 5, 3,
        0, 2, 0, 4, 1, 5, 0, 3], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 121262], edge_attr=[121262, 7], y=[32], num_nodes=6966, x=[6966], batch=[6966], ptr=[33])
pred=tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 0, 4, 4, 0, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0'), data.y=tensor([2, 0, 5, 2, 4, 4, 1, 5, 2, 4, 0, 1, 3, 5, 3, 3, 1, 1, 1, 3, 5, 5, 4, 5,
        2, 3, 4, 0, 4, 1, 4, 1], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 144698], edge_attr=[144698, 7], y=[32], num_nodes=7569, x

Iterating eval graphs:  17%|█▋        | 6/35 [00:00<00:02, 14.25batch/s]

DataBatch(edge_index=[2, 127950], edge_attr=[127950, 7], y=[32], num_nodes=7645, x=[7645], batch=[7645], ptr=[33])
pred=tensor([4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4,
        4, 4, 4, 0, 4, 4, 0, 4], device='cuda:0'), data.y=tensor([2, 4, 5, 4, 5, 0, 4, 1, 0, 0, 2, 1, 4, 4, 5, 4, 4, 4, 1, 1, 5, 1, 5, 4,
        0, 5, 4, 0, 1, 1, 0, 4], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 120872], edge_attr=[120872, 7], y=[32], num_nodes=7820, x=[7820], batch=[7820], ptr=[33])
pred=tensor([0, 4, 4, 4, 0, 4, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4], device='cuda:0'), data.y=tensor([3, 1, 0, 2, 2, 3, 5, 4, 0, 5, 4, 2, 4, 4, 1, 0, 1, 4, 0, 4, 0, 4, 5, 0,
        5, 2, 3, 0, 2, 5, 4, 0], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 125522], edge_attr=[125522, 7], y=[32], num_nodes=6715, x

Iterating eval graphs:  23%|██▎       | 8/35 [00:00<00:01, 14.20batch/s]

...

DataBatch(edge_index=[2, 129794], edge_attr=[129794, 7], y=[32], num_nodes=6955, x=[6955], batch=[6955], ptr=[33])
pred=tensor([4, 0, 4, 4, 0, 4, 4, 0, 4, 0, 1, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4, 1, 4, 2,
        0, 4, 4, 0, 4, 4, 4, 1], device='cuda:0'), data.y=tensor([1, 0, 4, 3, 1, 0, 3, 5, 4, 4, 0, 4, 0, 4, 4, 0, 4, 4, 1, 3, 4, 1, 3, 0,
        4, 4, 4, 2, 3, 5, 4, 1], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>


Iterating eval graphs:  91%|█████████▏| 32/35 [00:02<00:00, 12.68batch/s]

DataBatch(edge_index=[2, 154384], edge_attr=[154384, 7], y=[32], num_nodes=8229, x=[8229], batch=[8229], ptr=[33])
pred=tensor([1, 2, 4, 4, 4, 0, 1, 1, 0, 1, 0, 4, 4, 4, 4, 4, 2, 4, 4, 4, 0, 4, 4, 0,
        0, 4, 4, 4, 4, 2, 1, 4], device='cuda:0'), data.y=tensor([4, 3, 0, 3, 5, 1, 1, 1, 5, 3, 0, 1, 5, 4, 4, 3, 2, 4, 2, 2, 3, 5, 3, 0,
        0, 4, 3, 4, 3, 3, 4, 2], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 118838], edge_attr=[118838, 7], y=[32], num_nodes=6812, x=[6812], batch=[6812], ptr=[33])
pred=tensor([1, 4, 1, 4, 4, 1, 2, 4, 4, 4, 4, 1, 0, 4, 4, 1, 4, 0, 4, 0, 0, 0, 1, 1,
        4, 4, 1, 0, 1, 4, 4, 4], device='cuda:0'), data.y=tensor([3, 4, 0, 0, 4, 2, 5, 2, 4, 1, 3, 1, 1, 1, 3, 0, 2, 2, 4, 2, 0, 5, 1, 3,
        4, 4, 2, 1, 1, 3, 4, 5], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 153150], edge_attr=[153150, 7], y=[32], num_nodes=7683, x

Iterating eval graphs: 100%|██████████| 35/35 [00:02<00:00, 12.76batch/s]
Epoch 10/10, Loss: 1.6917, Train Acc: 0.3449, Val Acc: 0.3589
Epoch 10/10, Loss: 1.6917, Train Acc: 0.3449, Val Acc: 0.3589
Epoch 10/10, Loss: 1.6917, Train Acc: 0.3449, Val Acc: 0.3589
Epoch 10/10, Loss: 1.6917, Train Acc: 0.3449, Val Acc: 0.3589
Epoch 10/10, Loss: 1.6917, Train Acc: 0.3449, Val Acc: 0.3589


DataBatch(edge_index=[2, 158418], edge_attr=[158418, 7], y=[32], num_nodes=8101, x=[8101], batch=[8101], ptr=[33])
pred=tensor([1, 1, 0, 4, 4, 4, 4, 1, 4, 4, 1, 0, 4, 0, 0, 1, 4, 4, 4, 0, 0, 0, 2, 4,
        4, 4, 4, 4, 4, 0, 2, 4], device='cuda:0'), data.y=tensor([1, 2, 1, 5, 3, 4, 4, 2, 0, 5, 4, 4, 4, 2, 3, 0, 4, 4, 2, 4, 0, 0, 4, 0,
        2, 0, 4, 4, 4, 4, 5, 3], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
DataBatch(edge_index=[2, 145944], edge_attr=[145944, 7], y=[32], num_nodes=8005, x=[8005], batch=[8005], ptr=[33])
pred=tensor([0, 4, 0, 4, 4, 4, 0, 1, 0, 4, 2, 0, 1, 0, 4, 0, 4, 4, 4, 1, 0, 4, 4, 1,
        0, 0, 4, 1, 4, 4, 4, 4], device='cuda:0'), data.y=tensor([0, 3, 4, 4, 4, 5, 0, 1, 2, 3, 1, 4, 1, 0, 5, 0, 3, 3, 5, 4, 5, 2, 4, 1,
        1, 0, 4, 4, 5, 4, 2, 5], device='cuda:0')
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'torch.Tensor'>
Epoch 10/10, Loss: 1.6917, Train Acc: 0.3449, Val Acc: 0.3589
Best model updated a

In [44]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

11739

In [53]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [47]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=True)
save_predictions(predictions, args.test_path)

Iterating eval graphs:   0%|          | 0/74 [00:00<?, ?batch/s]


AttributeError: 'bool' object has no attribute 'sum'

In [ ]:
# test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)

# def evaluate(data_loader, model, device, calculate_accuracy=False):
#     model.eval()
#     correct = 0
#     total = 0
#     predictions = []
#     total_loss = 0
#     criterion = torch.nn.CrossEntropyLoss()
#     with torch.no_grad():
#         for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
#             print(data)
#             data = data.to(device)
#             output = model(data)
#             pred = output.argmax(dim=1)
            
#             if calculate_accuracy:
#                 print(f"{pred=}, {data.y=}")
#                 print(f"{type(pred)=}, {type(data.y)=}")
#                 # print(f"{pred data. }")

#                 correct += (pred == data.y).sum().item()
#                 total += data.y.size(0)
#                 total_loss += criterion(output, data.y).item()
#             else:
#                 predictions.extend(pred.cpu().numpy())
#     if calculate_accuracy:
#         accuracy = correct / total
#         return  total_loss / len(data_loader),accuracy
#     return predictions

# model.load_state_dict(torch.load(checkpoint_path))
# predictions = evaluate(test_loader, model, device, calculate_accuracy=True)
# save_predictions(predictions, args.test_path)

Iterating eval graphs:   0%|          | 0/74 [00:00<?, ?batch/s]

DataBatch(edge_index=[2, 150304], edge_attr=[150304, 7], num_nodes=7701, x=[7701], batch=[7701], ptr=[33])
pred=tensor([2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 3, 2, 2, 2, 2, 2, 2], device='cuda:0'), data.y=None
type(pred)=<class 'torch.Tensor'>, type(data.y)=<class 'NoneType'>


AttributeError: 'bool' object has no attribute 'sum'